## Integrate the particles where they feel all NBody forces

In [1]:
import tstrippy
from gcs import path_handler as ph
import gcs
import os 

In [2]:
GCname              =   "NGC104"
montecarlokey       =   "monte-carlo-000"
internal_dynamics   =   "isotropic-plummer"
potential           =   "pouliasis2017pii-GCNBody"
NP                  =   int(1e5)

load the particle positions

In [3]:
fname = ph.ParticleInitialConditions(GCname)
x,y,z,vx,vy,vz=gcs.extractors.ParticleInitialConditions.load_particles(fname,internal_dynamics,montecarlokey,NP)

In [12]:
# now get the positions at the begining of the orbit 
orbit_file_name = ph.GC_orbits(potential,GCname)
tH,xH,yH,zH,vxH,vyH,vzH=gcs.extractors.GCOrbits.extract_whole_orbit(orbit_file_name,montecarlokey)
t0,x0,y0,z0,vx0,vy0,vz0=gcs.extractors.GCOrbits.initial_simulation_coordinates(orbit_file_name,montecarlokey)

In [4]:
GCnames=list(tstrippy.Parsers.baumgardtMWGCs().data['Cluster'][:])
GCnames.remove(GCname)

In [13]:
ts,xs,ys,zs,vxs,vys,vzs=gcs.extractors.GCOrbits.extract_orbits_from_all_GCS(GCnames,potential,montecarlokey)

In [15]:
Masses,rh_mes,RAes,DECes,Rsunes,RVes,mualphaes,mu_deltaes=gcs.extractors.MonteCarloObservables.extract_all_GC_observables(GCnames,montecarlokey)

In [11]:
fname = ph.ParticleInitialConditions(GCname)
x,y,z,vx,vy,vz=gcs.extractors.ParticleInitialConditions.load_particles(fname,internal_dynamics,montecarlokey,NP)